**Course website**: http://lagex.github.io/geofisica2

**Note**: This notebook is part of the course "Geofísica 2" of Geology program of the 
[Universidade do Estado do Rio de Janeiro](http://www.uerj.br/). 
All content can be freely used and adapted under the terms of the 
[Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)

Esse documento que você está usando é um [IPython notebook](http://ipython.org/notebook.html). É um documento interativo que mistura texto (como esse), código (como abaixo), e o resultado de executar o código (que pode ser números, texto, figuras, videos, etc).

# Prática 6 - Sísmica de reflexão: CMP, NMO e empilhamento

Nessa prática, vamos simular a aquisição de uma seção Common Mid Point (CMP). Vamos ver como utilizar a correção de Normal Moveout (NMO) para determinar a velocidade. Também vamos ver como o empilhamento melhora a razão sinal-ruído dos nossos dados.

Para isso, vamos utilizar as simulações de ondas da biblioteca [Fatiando a Terra](http://www.fatiando.org). Essas simulações utilizam o [método de diferenças finitas](http://en.wikipedia.org/wiki/Finite_difference_method) para calcular soluções da equação da onda.

As tarefas abaixo acompanham as questões que vocês devem responder. Usem as simulações abaixo para te ajudar a responder as perguntas.

## Preparação

Rode a célula abaixo para carregar as componentes necessárias para fazer as simulações. Não se preocupe se aparecer um `:0: FutureWarning: IPython widgets are experimental and may change in the future.` abaixo. Isso é consequência de utilizar tecnologia de ponta.

In [ ]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from IPython.html import widgets
from IPython import display
from fatiando import utils
from fatiando.seismic.wavefd import Ricker, ElasticSH
from fatiando.vis import mpl
import fatiando
from multiprocessing import Pool

In [ ]:
print(fatiando.__version__)

## Simulação de um CMP para um modelo de duas camadas

Rode as células abaixo para rodar simulações de propagação de ondas em um modelo de duas camadas. Vamos utilizar essas simulações para extrair uma seção Common Mid Point (CMP) para trabalharmos.

A célula abaixo cria nosso modelo de duas camadas.

In [ ]:
shape = (150, 200)
spacing = 10
extent = [0, shape[1]*spacing, shape[0]*spacing, 0]
densidade = np.ones(shape)*1600
velocidade = np.ones(shape)*4000
l1 = shape[0]//3
densidade[l1:,:] = 1800
velocidade[l1:,:] = 5000
l2 = shape[0]//2
densidade[l2:,:] = 2000
velocidade[l2:,:] = 6000

Em seguida, precisamos definir onde serão localizadas as fontes e os receptores em nossa simulação. Vamos aproveitar e calcular também os espassamentos (offsets) dos receptores. Lembre-se: offset é a distância da conte ao receptor.

In [ ]:
fontes = np.array(list(reversed(range(55, shape[1]//2 - 5, 5))))
recep = np.array([shape[1] - s for s in fontes])
offsets = (recep - fontes)*spacing
print("Utilizando {} fontes e {} receptores.".format(len(fontes), len(recep)))
print('Fontes:     {}'.format(fontes*spacing))
print('Receptores: {}'.format(recep*spacing))
print('Offsets:    {}'.format(offsets))

Vamos rodar as simulações que precisamos (uma por fonte). **A barra de progresso não irá aparecer** pois vamos rodar as simulações em paralelo para agilizar o processo.

In [ ]:
def run_sim(fonte, its=800):    
    sim = ElasticSH(velocidade, densidade, spacing=spacing, taper=0.005, padding=45, verbose=False)
    sim.add_point_source((0, fonte), Ricker(5, 45, 2/45))
    sim.run(its)
    return sim

In [ ]:
print('Simulando...')
pool = Pool()
sims = pool.map(run_sim, fontes)
pool.close()
print('Terminado.')

Rode a célula abaixo para criarmos uma animação da primeira fonte (só para termos uma idéia do que está acontecendo).

In [ ]:
sims[0].animate(20, embed=True, dpi=50, fps=5, cutoff=0.0002, cmap='Greys')

Agora podemos extrair nosso CMP das simulações que fizemos. Vamos mostrá-lo como uma seção sísmica em escala de cinza.

In [ ]:
dt = sims[0].dt
times = np.linspace(0, dt*sims[0].size, sims[0].size)
CMP = np.empty((sims[0].size, len(recep)))
for i, sim in enumerate(sims):
    CMP[:, i] = sim[:, 0, recep[i]]

plt.figure(figsize=(6, 6))
plt.title("CMP")
mpl.seismic_image(CMP, dt=sims[0].dt, vmin=-0.00001, vmax=0.00001, aspect=20)
plt.xlabel(u'# traço')
plt.ylabel('tempo (s)')
plt.tight_layout()

## Correção de NMO e  análise de  velocidades

Agora que temos nosso CMP, podemos aplicar a correção de NMO e fazer a nossa análise de velocidades. Rode a célula abaixo para produzir uma figura interativa. Nessa figura, você poderá controlar qual é a velocidade utilizada na correção NMO do nosso CMP. **Determine a velocidade de NMO das duas reflexões**.

In [ ]:
def nmo_correction(CMP, times, offsets, v):
    nmo = np.zeros_like(CMP)
    for i, t0 in enumerate(times):
        for j, o in enumerate(offsets):
            t = np.sqrt(t0**2 + o**2/v[i]**2)
            k = t//dt
            if k < times.size - 1:
                # Linear interpolation of the amplitude
                y0, y1 = CMP[k, j], CMP[k + 1, j]
                x0, x1 = times[k], times[k + 1]
                nmo[i, j] = y0 + (y1 - y0)*(t - x0)/(x1 - x0)
    return nmo

def nmo_analise(v1, v2):
    v = v1*np.ones_like(times)
    v[times > 0.35] = v2
    nmo = nmo_correction(CMP, times, offsets, v)
    plt.figure(figsize=(7, 6))
    plt.subplot(121)
    plt.title('CMP')
    gain = 0.00001
    mpl.seismic_image(CMP, dt=sims[0].dt, vmin=-gain, vmax=gain, aspect=30)
    plt.grid()
    plt.xlabel(u'# traço')
    plt.ylabel('tempo (s)')
    plt.subplot(122)
    plt.title('Corrigido de NMO')
    gain = 0.00001
    mpl.seismic_image(nmo, dt=sims[0].dt, vmin=-gain, vmax=gain, aspect=30)
    plt.grid()
    plt.xlabel(u'# traço')
    plt.tight_layout()
widgets.interactive(nmo_analise, 
                    v1=widgets.FloatSliderWidget(min=2000, max=6000, step=100, value=2000),
                    v2=widgets.FloatSliderWidget(min=2000, max=6000, step=100, value=2000))

## Empilhamento

Após feita a correção de NMO, podemos então empilhar nosso CMP para produzir um único sismograma (traço) de incidência normal. Em sua forma mais simples, o empilhamento (stack) é a soma de todos os sismogramas (traços).

Antes disso, vamos entender melhor como funciona o empilhamento com um exemplo.

### Exemplo de empilhamento com dados aleatórios

Abaixo, vamos gerar sequências de dados totalmente aleatórios.

In [ ]:
dados_ruido = np.array([np.random.uniform(-0.3, 0.3, size=200) for i in range(10)])

Agora, rode a célula abaixo para produzir uma figura interativa. Nessa figura, você vai controlar a amplitude e localização de um **sinal não-aleatório** que vamos inserir nos dados. A figura mostra os dados a esquerda e o resultado do empilhamento a direita.

**Varie a posição e tamanho (amplitude) do sinal não-aleatório. Como o empilhamento afeta a qualidade dos dados e nossa habilidade de ver o sinal não-aleatório?**. 

In [ ]:
def empilhamento(pos_sinal, tamanho_sinal):
    N = dados_ruido.shape[1]
    x = np.arange(N)
    sinal = tamanho_sinal*utils.gaussian(x, pos_sinal, 2)
    dados = dados_ruido + sinal
    plt.figure(figsize=(10, 6))
    plt.subplot(121)
    plt.title(u'Dados com ruído e um sinal não-aleatório')
    for i, d in enumerate(dados):
        plt.plot(d + i + 1, x, '-k')
    plt.xlim(0, len(dados) + 1)
    plt.xlabel('# do dado')
    plt.grid()
    ax = plt.subplot(143)
    plt.title('Empilhamento')
    plt.plot(dados.sum(0), x, '-k')
    plt.grid()
    plt.xlim(-10, 10)
    plt.tight_layout()
widgets.interactive(empilhamento, pos_sinal=(0, dados_ruido.shape[1], 10), 
                    tamanho_sinal=widgets.FloatSliderWidget(min=0, max=1, step=0.1, value=0))

### Empilhamento do CMP com ruído

Finalmente, vamos empilhar nosso CMP. Mas para ter um pouco de graça, vamos primeiro colocar ruído aleatório nos dados para ficar mais parecido  com a realidade. 

Rode a célula abaixo para colocar ruído em nosso CMP.

In [ ]:
ruido = 0.00003
CMP_ruido = CMP + np.random.uniform(-ruido, ruido, CMP.shape)

plt.figure(figsize=(6, 6))
plt.title("CMP")
mpl.seismic_image(CMP_ruido, dt=sims[0].dt, vmin=-0.00001, vmax=0.00001, aspect=20)
plt.xlabel(u'# traço')
plt.ylabel('tempo (s)')
plt.tight_layout()

Agora que temos um CMP com ruído, vamos fazer novamente a análise de velocidades. Dessa vez, vamos também empilhar o CMP corrigido de NMO para ver como fica o resultado do empilhamento quando utilizamos  a velocidade  de NMO correta. Para comparação, vamos plotar ao lado um dos traços da seção corrigida de NMO (sem empilhamento).

Rode a célula abaixo para gerar a figura interativa para nossa análise de velocidades e empilhamento.

In [ ]:
def nmo_empilhamento(v1, v2):
    v = v1*np.ones_like(times)
    v[times > 0.35] = v2
    nmo = nmo_correction(CMP_ruido, times, offsets, v)
    plt.figure(figsize=(11, 6))
    plt.subplot(141)
    plt.title('CMP')
    gain = 0.00005
    mpl.seismic_image(CMP_ruido, dt=sims[0].dt, vmin=-gain, vmax=gain, aspect=40)
    plt.grid(True)
    plt.ylabel('tempo (s)')
    plt.subplot(142)
    plt.title('Corrigido de NMO')
    gain = 0.00005
    mpl.seismic_image(nmo, dt=sims[0].dt, vmin=-gain, vmax=gain, aspect=40)
    plt.grid(True)
    plt.subplot(143)
    plt.title('Empilhamento')
    stack = np.atleast_2d(nmo.sum(1)).T
    mpl.seismic_wiggle(stack, dt=dt, scale=500)
    plt.xlim(-1, 1)
    plt.grid()
    plt.subplot(144)
    plt.title(u'Primeiro traço do NMO')
    mpl.seismic_wiggle(np.atleast_2d(nmo[:, 0]).T, dt=dt, scale=5000)
    plt.xlim(-1, 1)
    plt.grid()
    plt.tight_layout()
    return nmo
w = widgets.interactive(nmo_empilhamento, 
                        v1=widgets.FloatSliderWidget(min=2000, max=6000, step=100, value=2000),
                        v2=widgets.FloatSliderWidget(min=2000, max=6000, step=100, value=2000))
w